In [10]:
from IPython.display import display

import pandas as pd
import numpy as np

from utils import load_dataset, clean_dataset
from bias_utils import aequitas_preprocessing, df_to_aequitas, fairness_report


from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness

In [2]:
df = load_dataset()
df = clean_dataset(df) # select only searched entries, location preprocessed

In [3]:
display(df.head(2))
display(df.shape)

,VehicleSearchedIndicator,ContrabandIndicator,Department Name,InterventionDateTime,InterventionLocationName,InterventionReasonCode,ReportingOfficerIdentificationID,ResidentIndicator,SearchAuthorizationCode,StatuteReason,SubjectAge,SubjectEthnicityCode,SubjectRaceCode,SubjectSexCode,TownResidentIndicator
71,True,False,Bridgeport,2013-10-01 00:46:00,bridgeport,V,1207,True,I,Speed Related,37.0,H,W,M,True
143,True,True,Milford,2013-10-01 01:50:00,milford,E,2325,True,I,Defective Lights,30.0,N,W,M,True


(76743, 15)

In [7]:
def departments_fairness_report(df: pd.DataFrame, size_threshold = 100) -> list:
    """
    Evaluate fairness for all departments.
    Only account for those who have at least <size_threshold>
    instances of searched vehicles.
    """
    _df = df.copy()
    departments_clean_fairness = []
    departments_ignored_size = []
    failed_fairness_departments = {}
    report = {}
    
    protected_classes = ['SubjectAge','SubjectRaceCode','SubjectSexCode','Is State Resident?', 'Is Town Resident?']
    for department in _df['Department Name'].unique():
        print(department)
        department_df = _df[_df['Department Name'] == department].copy()
        if department_df.shape[0] < size_threshold:
            departments_ignored_size.append(department)
            continue
            
        department_df = aequitas_preprocessing(department_df)
        aequitas_df = df_to_aequitas(department_df,'VehicleSearchedIndicator', protected_classes)
        try:
            fairness_df,fairness_text_report = fairness_report(aequitas_df,text_report=True)
        except Exception as e:
            print(e)
            return aequitas_df
        
        if fairness_df.shape[0] == 0:
            departments_clean_fairness.append(department)
        else:
            failed_fairness_departments[department] = {'text_report':fairness_text_report, 'df': fairness_df}
    
    report = {'departments_clean_fairness':departments_clean_fairness,
             'departments_ignored_size':departments_ignored_size,
             'failed_fairness_departments':failed_fairness_departments}
    return report
            

In [8]:
result = departments_fairness_report(df)

Bridgeport
model_id, score_thresholds 1 {'rank_abs': [1562]}
get_disparity_predefined_group()
Milford
model_id, score_thresholds 1 {'rank_abs': [1224]}
get_disparity_predefined_group()
Torrington
model_id, score_thresholds 1 {'rank_abs': [663]}
get_disparity_predefined_group()
State Police
model_id, score_thresholds 1 {'rank_abs': [5449]}
get_disparity_predefined_group()
Greenwich
model_id, score_thresholds 1 {'rank_abs': [601]}
get_disparity_predefined_group()
Westport
model_id, score_thresholds 1 {'rank_abs': [837]}
get_disparity_predefined_group()
Middletown
model_id, score_thresholds 1 {'rank_abs': [1178]}
get_disparity_predefined_group()
Naugatuck
model_id, score_thresholds 1 {'rank_abs': [921]}
get_disparity_predefined_group()
West Hartford
model_id, score_thresholds 1 {'rank_abs': [2214]}
get_disparity_predefined_group()
Norwalk
model_id, score_thresholds 1 {'rank_abs': [1762]}
get_disparity_predefined_group()
Danbury
model_id, score_thresholds 1 {'rank_abs': [1588]}
get_dispari

In [41]:
g = Group();
xtab, _ = g.get_crosstabs(result);

size_threshold = 20
filtered_xtab = xtab[xtab.group_size > size_threshold]

# Bias through disparity scores
b = Bias();

bdf = b.get_disparity_major_group(filtered_xtab,
                                original_df = result,
                                label_score_ref='fdr')

# Fairness through 0.8 threshold on disparity
f = Fairness(tau=0.8);
fdf = f.get_group_value_fairness(bdf);

failed_fairness = fdf[fdf['FDR Parity'] == False]

important_columns = ['attribute_name', 'attribute_value','fdr_disparity','fdr_ref_group_value','FDR Parity']

# return text report of failed fairness
text_report = True
if text_report:
    report = "\nFairness Report\n----------\n"
    for idx,row in failed_fairness.iterrows():
        text = f'{row.attribute_name} : {row.attribute_value}\n'
        text += f'It is {round(row.fdr_disparity,2)} times more likely to fail in people who are {row.attribute_value} than people who are {row.fdr_ref_group_value}.\n\n'
        report += text

model_id, score_thresholds 1 {'rank_abs': [127]}
get_disparity_major_group()


In [45]:
fdf[['attribute_name', 'attribute_value','fdr_disparity','fdr_ref_group_value','FDR Parity']]

,attribute_name,attribute_value,fdr_disparity,fdr_ref_group_value,FDR Parity
0,SubjectAge,21-45,1.000000,21-45,True
1,SubjectRaceCode,Black,1.183575,White,True
2,SubjectRaceCode,White,1.000000,White,True
3,SubjectSexCode,Female,1.066406,Male,True
4,SubjectSexCode,Male,1.000000,Male,True
5,Is State Resident?,Yes,1.000000,Yes,True
6,Is Town Resident?,No,1.000000,No,True


In [39]:
filtered_xtab[['attribute_name','attribute_value','fdr','group_size']]

,attribute_name,attribute_value,fdr,group_size
0,SubjectAge,21-45,0.723404,94
4,SubjectRaceCode,Black,0.777778,54
6,SubjectRaceCode,White,0.657143,70
7,SubjectSexCode,Female,0.750000,36
8,SubjectSexCode,Male,0.703297,91
9,Is State Resident?,Yes,0.716535,127
10,Is Town Resident?,No,0.724138,116


In [31]:
size_threshold = 20
filtered_xtab = xtab[xtab.group_size > size_threshold]

Index(['model_id', 'score_threshold', 'k', 'attribute_name', 'attribute_value',
       'tpr', 'tnr', 'for', 'fdr', 'fpr', 'fnr', 'npv', 'precision', 'pp',
       'pn', 'ppr', 'pprev', 'fp', 'fn', 'tn', 'tp', 'group_label_pos',
       'group_label_neg', 'group_size', 'total_entities', 'prev'],
      dtype='object')

In [32]:
xtab[['attribute_name','attribute_value','fdr','group_size']]

,attribute_name,attribute_value,fdr,group_size
0,SubjectAge,21-45,0.723404,94
1,SubjectAge,<21,0.600000,15
2,SubjectAge,>45,0.777778,18
3,SubjectRaceCode,Asian,1.000000,2
4,SubjectRaceCode,Black,0.777778,54
5,SubjectRaceCode,Native,1.000000,1
6,SubjectRaceCode,White,0.657143,70
7,SubjectSexCode,Female,0.750000,36
8,SubjectSexCode,Male,0.703297,91
9,Is State Resident?,Yes,0.716535,127


In [18]:
failed_fairness

,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,...,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
10,1,binary 0/1,127,SubjectAge,>45,1.0,0.0,NaN,0.777778,1.0,...,NaN,True,NaN,NaN,True,False,NaN,True,True,False
11,1,binary 0/1,127,SubjectAge,>45,1.0,0.0,NaN,0.777778,1.0,...,NaN,True,NaN,NaN,True,False,NaN,True,True,False
12,1,binary 0/1,127,SubjectAge,>45,1.0,0.0,NaN,0.777778,1.0,...,NaN,True,NaN,NaN,True,False,NaN,True,True,False
13,1,binary 0/1,127,SubjectAge,>45,1.0,0.0,NaN,0.777778,1.0,...,NaN,True,NaN,NaN,True,False,NaN,True,True,False
14,1,binary 0/1,127,SubjectAge,>45,1.0,0.0,NaN,0.777778,1.0,...,NaN,True,NaN,NaN,True,False,NaN,True,True,False


In [15]:
print(report)


Fairness Report
----------
SubjectAge : >45
It is 1.3 times more likely to fail in people who are >45 than people who are <21.

SubjectAge : >45
It is 1.3 times more likely to fail in people who are >45 than people who are <21.

SubjectAge : >45
It is 1.3 times more likely to fail in people who are >45 than people who are <21.

SubjectAge : >45
It is 1.3 times more likely to fail in people who are >45 than people who are <21.

SubjectAge : >45
It is 1.3 times more likely to fail in people who are >45 than people who are <21.




In [6]:
milford['departments_clean_fairness']

NameError: name 'milford' is not defined

In [25]:
milford.columns

Index(['SubjectAge', 'SubjectRaceCode', 'SubjectSexCode', 'Is State Resident?',
       'Is Town Resident?', 'label_value', 'score', 'label_value', 'score'],
      dtype='object')